In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip "/content/drive/MyDrive/UROP/COMP4901V Project/height_coco_dataset.zip"

串流輸出內容已截斷至最後 5000 行。
  inflating: images/S_NYC_3150_63840.png  
  inflating: images/N_NYC_24780_34650.png  
  inflating: images/W_NYC_26040_14910.png  
  inflating: images/W_NYC_11760_21630.png  
  inflating: images/S_NYC_30450_19950.png  
  inflating: images/W_NYC_5880_28770.png  
  inflating: images/S_NYC_17430_32970.png  
  inflating: images/W_NYC_15540_38430.png  
  inflating: images/S_NYC_8190_46620.png  
  inflating: images/N_NYC_45990_56280.png  
  inflating: images/N_NYC_26670_13020.png  
  inflating: images/N_NYC_55020_37380.png  
  inflating: images/W_NYC_40740_6090.png  
  inflating: images/W_NYC_2520_19740.png  
  inflating: images/S_NYC_3990_56700.png  
  inflating: images/N_NYC_51450_43470.png  
  inflating: images/W_NYC_18690_14490.png  
  inflating: images/N_NYC_37590_51240.png  
  inflating: images/S_NYC_16800_26040.png  
  inflating: images/N_NYC_12390_32340.png  
  inflating: images/S_NYC_43680_9870.png  
  inflating: images/W_NYC_15750_37800.png  
  inflating: image

In [ ]:
!git clone https://github.com/facebookresearch/detr.git   #cloning github repo of detr to import its unique loss

Cloning into 'detr'...
remote: Enumerating objects: 263, done.
remote: Total 263 (delta 0), reused 0 (delta 0), pack-reused 263
Receiving objects: 100% (263/263), 12.88 MiB | 14.76 MiB/s, done.
Resolving deltas: 100% (122/122), done.


In [ ]:
import os
import numpy as np 
import pandas as pd 
from datetime import datetime
import time
import random
from tqdm.autonotebook import tqdm


#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler

#sklearn
from sklearn.model_selection import StratifiedKFold

#CV
import cv2

################# DETR FUCNTIONS FOR LOSS######################## 
import sys
sys.path.append('./detr/')

from detr.models.matcher import HungarianMatcher
from detr.models.detr import SetCriterion
#################################################################

#Albumenatations
import albumentations as A
import matplotlib.pyplot as plt
from albumentations.pytorch.transforms import ToTensorV2

#Glob
from glob import glob

<ipython-input-5-d875a0c87f69>:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


# Utils

* AverageMeter - class for averaging loss,metric,etc over epochs

In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

# Configuration

Basic configuration for this model

In [ ]:
n_folds = 5
seed = 42
num_classes = 2
num_queries = 100
null_class_coef = 0.5
BATCH_SIZE = 32
LR = 2e-5
EPOCHS = 2

# Seed Everything

Seeding everything for reproducible results

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
seed_everything(seed)

# Preparing the Data

* For preparation of data I use code from Alex's awesome kernel [here](https://www.kaggle.com/shonenkov/training-efficientdet)
* The data can be split into any number of folds as you want , split is stratified based on number of boxes and source

# Augmentations

* As suggested by aleksendra in her kernel ,augentations will play a major role and hence took her up advice and use awesome augmentations , cut-mix and other will be included in future versions

In [ ]:
def get_train_transforms():
    return A.Compose([A.OneOf([A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit= 0.2, val_shift_limit=0.2, p=0.9),
                               
                      A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.9)],p=0.9),
                      
                      A.ToGray(p=0.01),
                      
                      A.HorizontalFlip(p=0.5),
                      
                      A.VerticalFlip(p=0.5),
                      
                      A.Resize(height=512, width=512, p=1),
                      
                      A.Cutout(num_holes=8, max_h_size=64, max_w_size=64, fill_value=0, p=0.5),
                      
                      ToTensorV2(p=1.0)],
                      
                      p=1.0,
                     
                      bbox_params=A.BboxParams(format='coco',min_area=0, min_visibility=0,label_fields=['labels'])
                      )

def get_valid_transforms():
    return A.Compose([A.Resize(height=512, width=512, p=1.0),
                      ToTensorV2(p=1.0)], 
                      p=1.0, 
                      bbox_params=A.BboxParams(format='coco',min_area=0, min_visibility=0,label_fields=['labels'])
                      )

# Creating Dataset

* I hope you have the video by now , DETR accepts data in coco format which is (x,y,w,h)(for those who do not know there are two formats coco and pascal(smin,ymin,xmax,ymax) which are widely used) . So now we need to prepare data in that format

In [ ]:
from pycocotools.coco import COCO
from PIL import Image
import torchvision.transforms as T
import numpy as np

class COCODataset(Dataset):
    def __init__(self, root_dir, coco_annotation, transform=None):
        ...
        self.coco = COCO(coco_annotation)
        self.image_ids = self.coco.getImgIds()
        self.transform = transform
        self.root_dir = root_dir
        
    def __getitem__(self, idx, max_no_objs=100):
        image_id = self.image_ids[idx]
        file_id = self.coco.loadImgs(image_id)[0]['file_name'] # Get file ID from image metadata
        image = np.array(Image.open(os.path.join(self.root_dir, file_id)).convert('RGB')).astype(np.float32)
        image /= 255.0
        ann_ids = self.coco.getAnnIds(imgIds=image_id)
        anns = self.coco.loadAnns(ann_ids)

        # Initialize an empty array to store the boxes
        boxes = np.empty((0, 4))

        for ann in anns:
            box = list(ann['bbox'])
            if box[2] > 0.00001 and box[3] > 0.00001:
                # Append the box to the 2D array
                boxes = np.vstack((boxes, box))

        area = boxes[:,2]*boxes[:,3]
        labels = np.zeros(len(boxes))
        if self.transform is not None:
            sample = {
                'image': image,
                'bboxes': boxes,
                'labels': labels
            }
            sample = self.transform(**sample)
            image = sample['image']
            boxes = sample['bboxes']
            labels = sample['labels']
 
        target = {}
        _,h,w = image.shape
        target['boxes'] = torch.as_tensor(boxes,dtype=torch.float32)/h
        target['labels'] = torch.as_tensor(labels,dtype=torch.long)
        target['image_id'] = torch.tensor([image_id])
        target['area'] = torch.as_tensor(area, dtype=torch.float32)
        
        return torch.as_tensor(image,dtype=torch.float32), target, image_id
        
    def __len__(self):
        return len(self.image_ids)

# Model

* Initial DETR model is trained on coco dataset , which has 91 classes + 1 background class , hence we need to modify it to take our own number of classes
* Also DETR model takes in 100 queries ie ,it outputs total of 100 bboxes for every image , we can very well change that too

In [ ]:
class DETRModel(nn.Module):
    def __init__(self,num_classes,num_queries):
        super(DETRModel,self).__init__()
        self.num_classes = num_classes
        self.num_queries = num_queries
        
        self.model = torch.hub.load('facebookresearch/detr', 'detr_resnet50', pretrained=True)
        self.in_features = self.model.class_embed.in_features
        
        self.model.class_embed = nn.Linear(in_features=self.in_features,out_features=self.num_classes)
        self.model.num_queries = self.num_queries
        
    def forward(self,images):
        return self.model(images)

# Matcher and Bipartite Matching Loss

Now we make use of the unique loss that the model uses and for that we need to define the matcher. DETR calcuates three individual losses :
* Classification Loss for labels(its weight can be set by loss_ce)
* Bbox Loss (its weight can be set by loss_bbox)
* Loss for Background class

In [ ]:
'''
code taken from github repo detr , 'code present in engine.py'
'''

matcher = HungarianMatcher()

weight_dict = {'loss_ce': 1, 'loss_bbox': 1 , 'loss_giou': 1}

losses = ['labels', 'boxes', 'cardinality']

# Training Function

Training of DETR is unique and different from FasteRRcnn  and EfficientDET , as we train the criterion as well , the training function can be viewed here : https://github.com/facebookresearch/detr/blob/master/engine.py

In [ ]:
def train_fn(data_loader,model,criterion,optimizer,device,scheduler,epoch):
    model.train()
    criterion.train()
    
    summary_loss = AverageMeter()
    
    tk0 = tqdm(data_loader, total=len(data_loader))
    
    for step, (images, targets, image_ids) in enumerate(tk0):
        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        

        output = model(images)
        
        loss_dict = criterion(output, targets)
        weight_dict = criterion.weight_dict
        
        losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)
        
        optimizer.zero_grad()

        losses.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
        
        summary_loss.update(losses.item(),BATCH_SIZE)
        tk0.set_postfix(loss=summary_loss.avg)
        
    return summary_loss

# Eval Function

In [ ]:
def eval_fn(data_loader, model,criterion, device):
    model.eval()
    criterion.eval()
    summary_loss = AverageMeter()
    
    with torch.no_grad():
        
        tk0 = tqdm(data_loader, total=len(data_loader))
        for step, (images, targets, image_ids) in enumerate(tk0):
            
            images = list(image.to(device) for image in images)
            targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

            output = model(images)
        
            loss_dict = criterion(output, targets)
            weight_dict = criterion.weight_dict
        
            losses = sum(loss_dict[k] * weight_dict[k] for k in loss_dict.keys() if k in weight_dict)
            
            summary_loss.update(losses.item(),BATCH_SIZE)
            tk0.set_postfix(loss=summary_loss.avg)
    
    return summary_loss

# Engine

In [ ]:
def collate_fn(batch):
    return tuple(zip(*batch))

In [ ]:
# train_dataset = COCODataset(root_dir="images", coco_annotation="coco_train.json", transform=get_train_transforms())
# train_data_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, collate_fn=collate_fn)

# val_dataset = COCODataset(root_dir="images", coco_annotation="coco_val.json", transform=get_valid_transforms()) 
# valid_data_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, collate_fn=collate_fn)

# device = torch.device('cuda')
# model = DETRModel(num_classes=num_classes,num_queries=num_queries)
# model = model.to(device)
# criterion = SetCriterion(num_classes-1, matcher, weight_dict, eos_coef = null_class_coef, losses=losses)
# criterion = criterion.to(device)


# optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

# best_loss = 10**5
# for epoch in range(EPOCHS):
#     train_loss = train_fn(train_data_loader, model,criterion, optimizer,device,scheduler=None,epoch=epoch)
#     valid_loss = eval_fn(valid_data_loader, model,criterion, device)
    
#     print('|EPOCH {}| TRAIN_LOSS {}| VALID_LOSS {}|'.format(epoch+1,train_loss.avg,valid_loss.avg))
    
#     if valid_loss.avg < best_loss:
#         best_loss = valid_loss.avg
#         print('Best model found for Fold {} in Epoch {}........Saving Model'.format(epoch+1))
#         torch.save(model.state_dict(), f'detr_best.pth')

In [ ]:
!cp "/content/drive/MyDrive/UROP/COMP4901V Project/obj_detection/detr_best_32_new.pth" .

In [ ]:
!pip install albumentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from PIL import ImageOps

def view_sample(model,device, i):
    '''
    Code taken from Peter's Kernel 
    https://www.kaggle.com/pestipeti/pytorch-starter-fasterrcnn-train
    '''
    val_dataset = COCODataset(root_dir="images", coco_annotation="valid_coco.json", transform=get_valid_transforms()) 
    valid_data_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, collate_fn=collate_fn)
    
    images, targets, image_ids = next(iter(valid_data_loader))
    _,h,w = images[i].shape # for de normalizing images
    
    images = list(img.to(device) for img in images)
    targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

    building_index = targets[i]['building_index'].
    
    boxes = targets[i]['boxes'].cpu().numpy() *h
    boxes = [np.array(box).astype(np.int32) for box in boxes]
    
    sample = images[i].permute(1,2,0).cpu().numpy()
    
    model.eval()
    model.to(device)
    cpu_device = torch.device("cpu")
    
    with torch.no_grad():
        outputs = model(images)
        
    outputs = [{k: v.to(cpu_device) for k, v in outputs.items()}]
    
    fig, ax = plt.subplots(1, 1, figsize=(16, 8))

    for box in boxes:
        cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2]+box[0], box[3]+box[1]),
                  (220, 220, 0), 1)
        

    oboxes = outputs[0]['pred_boxes'][i].detach().cpu().numpy()
    oboxes = [np.array(box).astype(np.int32) for box in oboxes*480]
    prob   = outputs[0]['pred_logits'][i].softmax(1).detach().cpu().numpy()[:,0]
    index = output[0]['pred_building_index'][i].detach().cpu().numpy
    
    for box,p in zip(oboxes,prob):
        
        if p >0.5:
            color = (0,0,220) #if p>0.5 else (0,0,0)
            cv2.rectangle(sample,
                  (box[0], box[1]),
                  (box[2]+box[0], box[3]+box[1]),
                  color, 1)
    
    ax.set_axis_off()
    ax.imshow(sample)

In [ ]:
model = DETRModel(num_classes=num_classes,num_queries=num_queries)
model.load_state_dict(torch.load("/content/detr_best_32_new.pth"))
view_sample(model=model,device=torch.device('cuda'),i=8)

Using cache found in /root/.cache/torch/hub/facebookresearch_detr_main
